# E-Commerce Healthcare Orders Dataset

## Context
The dataset used for this notebook contains data of an healthcare company. The healthcare company faces an issue. There are high return rates of products in India. The goal of this notebook is to find patterns of why products are returned.

## Source
The dataset is from Kaggle and can be found on https://www.kaggle.com/adishgolechha/ecommerce-healthcare-orders-dataset.

## Description		
| Variable | Description |
| :- | :- |
| Serial Number | Serial number of product |
| ID | Unique Order ID assigned as key to every order |
| Name | Customer Name |
| City | City where the order is delivered |
| State | State where the order is delivered |
| Address | Delivery Address |
| isCOD | A Boolean of if the order is cash on delivery or not |
| Date Placed | Date the order is placed |
| Status | Status of the order - being Delivered or Returned |
| IVR | A confirmation of payment status, pre-paid orders are IVR confirmed |
| Remarks | Internal field, handled by Customer Service Agents |
| Total | Order total |
| Date Delivered | Date of delivery of product |
| Date Returned | Date where product was returned (only filled if returned) |
| PID | NaN |
| Category | Product category |
| Quantity | Quantatiy of ordered product |
| Product Name | Name of ordered product |

In [1]:
import os
import sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import csv
import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = [8,8]
sns.set_style("whitegrid")

## Data Import

In [4]:
path = r"data\OrdersCleaned.csv"
df = pd.read_csv(path, encoding = "ISO-8859-1")
df.head()

,Unnamed: 0,ID,Name,City,State,Address,isCOD,Date Placed,Status,IVR,Remarks,Total,Date Delivered,Date Returned,PID,Category,Quantity,Product Name
0,0,30145,Man,Nayagarh,Odisha,"Jagadish Prasad, Golapokhari-Bhapur Road, Fate...",True,2021-01-31 17:45:59,Delivered,Confirmed,pickrr,999,2021-02-14 04:37:00,NaN,32,WL,2,One Week Weight-Loss (Peach)
1,1,30144,Dik,Thane,Maharashtra,"Shanti Garden Sector 4 , flat 401, building 2,...",True,2021-01-31 17:33:57,Delivered,Confirmed,NaN,599,2021-02-02 23:19:18,NaN,23,D,2,One Week Detox Trial
2,2,30143,Shi,Bangalore,Karnataka,"#280 2nd main 2nd cross, vinobha nagar kg halli",True,2021-01-31 17:33:02,Delivered,Confirmed,On call,599,2021-02-08 00:08:25,NaN,23,D,2,One Week Detox Trial
3,3,30142,Pre,Mumbai,Maharashtra,"1404/ Accord Nidhi, above Bikaji, Link Road, M...",True,2021-01-31 17:31:57,Delivered,Confirmed,Script,3596,2021-02-09 03:17:08,NaN,34,WL,2,One Month Weight-Loss (Peach)
4,4,30138,Dr.,Pauri Garhwal,Uttarakhand,"10 am to 3 pm, Please call Before Coming, Room...",True,2021-01-31 17:06:55,Delivered,Confirmed,Script,999,2021-02-04 01:59:40,NaN,31,WL,2,One Week Weight-Loss (Mint)


In [8]:
# rename "Unnamed: 0" to "Serial Number"
df = df.rename(columns={df.columns[0]: "Serial Number"})
df.head()

,Serial Number,ID,Name,City,State,Address,isCOD,Date Placed,Status,IVR,Remarks,Total,Date Delivered,Date Returned,PID,Category,Quantity,Product Name
0,0,30145,Man,Nayagarh,Odisha,"Jagadish Prasad, Golapokhari-Bhapur Road, Fate...",True,2021-01-31 17:45:59,Delivered,Confirmed,pickrr,999,2021-02-14 04:37:00,NaN,32,WL,2,One Week Weight-Loss (Peach)
1,1,30144,Dik,Thane,Maharashtra,"Shanti Garden Sector 4 , flat 401, building 2,...",True,2021-01-31 17:33:57,Delivered,Confirmed,NaN,599,2021-02-02 23:19:18,NaN,23,D,2,One Week Detox Trial
2,2,30143,Shi,Bangalore,Karnataka,"#280 2nd main 2nd cross, vinobha nagar kg halli",True,2021-01-31 17:33:02,Delivered,Confirmed,On call,599,2021-02-08 00:08:25,NaN,23,D,2,One Week Detox Trial
3,3,30142,Pre,Mumbai,Maharashtra,"1404/ Accord Nidhi, above Bikaji, Link Road, M...",True,2021-01-31 17:31:57,Delivered,Confirmed,Script,3596,2021-02-09 03:17:08,NaN,34,WL,2,One Month Weight-Loss (Peach)
4,4,30138,Dr.,Pauri Garhwal,Uttarakhand,"10 am to 3 pm, Please call Before Coming, Room...",True,2021-01-31 17:06:55,Delivered,Confirmed,Script,999,2021-02-04 01:59:40,NaN,31,WL,2,One Week Weight-Loss (Mint)


## Initial EDA

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1590 entries, 0 to 1589
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Serial Number   1590 non-null   int64 
 1   ID              1590 non-null   int64 
 2   Name            1590 non-null   object
 3   City            1586 non-null   object
 4   State           1586 non-null   object
 5   Address         1590 non-null   object
 6   isCOD           1590 non-null   bool  
 7   Date Placed     1590 non-null   object
 8   Status          1590 non-null   object
 9   IVR             1590 non-null   object
 10  Remarks         919 non-null    object
 11  Total           1590 non-null   int64 
 12  Date Delivered  1401 non-null   object
 13  Date Returned   187 non-null    object
 14  PID             1555 non-null   object
 15  Category        1587 non-null   object
 16  Quantity        1590 non-null   int64 
 17  Product Name    1590 non-null   object
dtypes: bool(

The column "Date Returned" contains a lot of missing values. I think that this feature has no influence on the output, because when it should be predicted whether a product is returned or not, than it does not make sense to have a return date, because the product is not returned yet. Therefore, this feature is discarded. <br> <br>
The column "Remarks" also contains a lot of missing values. Let's later dive deeper into this feature and check, if we should discard this one or replace the missing values. <br> <br>
The columns "Date Delivered" and "Category" are also containing some missing values. Let's later try to impute them.

In [21]:
# let's drop date returned column
df_changed = df.drop(columns="Date Returned")
df_changed.head()

,Serial Number,ID,Name,City,State,Address,isCOD,Date Placed,Status,IVR,Remarks,Total,Date Delivered,PID,Category,Quantity,Product Name
0,0,30145,Man,Nayagarh,Odisha,"Jagadish Prasad, Golapokhari-Bhapur Road, Fate...",True,2021-01-31 17:45:59,Delivered,Confirmed,pickrr,999,2021-02-14 04:37:00,32,WL,2,One Week Weight-Loss (Peach)
1,1,30144,Dik,Thane,Maharashtra,"Shanti Garden Sector 4 , flat 401, building 2,...",True,2021-01-31 17:33:57,Delivered,Confirmed,NaN,599,2021-02-02 23:19:18,23,D,2,One Week Detox Trial
2,2,30143,Shi,Bangalore,Karnataka,"#280 2nd main 2nd cross, vinobha nagar kg halli",True,2021-01-31 17:33:02,Delivered,Confirmed,On call,599,2021-02-08 00:08:25,23,D,2,One Week Detox Trial
3,3,30142,Pre,Mumbai,Maharashtra,"1404/ Accord Nidhi, above Bikaji, Link Road, M...",True,2021-01-31 17:31:57,Delivered,Confirmed,Script,3596,2021-02-09 03:17:08,34,WL,2,One Month Weight-Loss (Peach)
4,4,30138,Dr.,Pauri Garhwal,Uttarakhand,"10 am to 3 pm, Please call Before Coming, Room...",True,2021-01-31 17:06:55,Delivered,Confirmed,Script,999,2021-02-04 01:59:40,31,WL,2,One Week Weight-Loss (Mint)


Now, the pandas function "describe" is called. This is always nice to get the first insights into the numerical features and their value ranges and some statistics.

In [22]:
df_changed.describe()

,Serial Number,ID,Total,Quantity
count,1590.000000,1590.000000,1590.000000,1590.000000
mean,794.500000,28909.873585,1762.896855,1.788679
std,459.137779,724.640062,1227.129226,0.655612
min,0.000000,27634.000000,0.000000,1.000000
25%,397.250000,28282.250000,799.000000,1.000000
50%,794.500000,28912.500000,999.000000,2.000000
75%,1191.750000,29537.750000,2876.000000,2.000000
max,1589.000000,30145.000000,7992.000000,12.000000


Seems like Serial Number and ID columns are only containing unique values. Let's check that and drop them in case that's true, because columns with only unique values are not delivering any usefull features for machine learning.

In [23]:
for col in df.columns:
    print(f"Column {col} contains {len(df[col].unique())} unique values ({ 100 * len(df[col].unique()) / df.shape[0]}% of total data).")

Column Serial Number contains 1590 unique values (100.0% of total data).
Column ID contains 1590 unique values (100.0% of total data).
Column Name contains 532 unique values (33.459119496855344% of total data).
Column City contains 306 unique values (19.245283018867923% of total data).
Column State contains 36 unique values (2.2641509433962264% of total data).
Column Address contains 1557 unique values (97.9245283018868% of total data).
Column isCOD contains 2 unique values (0.12578616352201258% of total data).
Column Date Placed contains 1578 unique values (99.24528301886792% of total data).
Column Status contains 3 unique values (0.18867924528301888% of total data).
Column IVR contains 2 unique values (0.12578616352201258% of total data).
Column Remarks contains 63 unique values (3.9622641509433962% of total data).
Column Total contains 115 unique values (7.232704402515723% of total data).
Column Date Delivered contains 1397 unique values (87.86163522012579% of total data).
Column Da

Okay. The columns "Serial Number" and "ID" can be dropped, because they are only containing unique values. The column address is also containing a lot of unique values, but this column could be used to parse out usefull data. The date columns are important for the time series analysis!

In [24]:
df_changed = df_changed.drop(columns=["ID", "Serial Number"])
df_changed.head()

,Name,City,State,Address,isCOD,Date Placed,Status,IVR,Remarks,Total,Date Delivered,PID,Category,Quantity,Product Name
0,Man,Nayagarh,Odisha,"Jagadish Prasad, Golapokhari-Bhapur Road, Fate...",True,2021-01-31 17:45:59,Delivered,Confirmed,pickrr,999,2021-02-14 04:37:00,32,WL,2,One Week Weight-Loss (Peach)
1,Dik,Thane,Maharashtra,"Shanti Garden Sector 4 , flat 401, building 2,...",True,2021-01-31 17:33:57,Delivered,Confirmed,NaN,599,2021-02-02 23:19:18,23,D,2,One Week Detox Trial
2,Shi,Bangalore,Karnataka,"#280 2nd main 2nd cross, vinobha nagar kg halli",True,2021-01-31 17:33:02,Delivered,Confirmed,On call,599,2021-02-08 00:08:25,23,D,2,One Week Detox Trial
3,Pre,Mumbai,Maharashtra,"1404/ Accord Nidhi, above Bikaji, Link Road, M...",True,2021-01-31 17:31:57,Delivered,Confirmed,Script,3596,2021-02-09 03:17:08,34,WL,2,One Month Weight-Loss (Peach)
4,Dr.,Pauri Garhwal,Uttarakhand,"10 am to 3 pm, Please call Before Coming, Room...",True,2021-01-31 17:06:55,Delivered,Confirmed,Script,999,2021-02-04 01:59:40,31,WL,2,One Week Weight-Loss (Mint)


In [29]:
df_changed['Date Placed'] =  pd.to_datetime(df_changed['Date Placed'], infer_datetime_format=True)
df_changed['Date Delivered'] =  pd.to_datetime(df_changed['Date Delivered'], infer_datetime_format=True)

In [30]:
df_changed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1590 entries, 0 to 1589
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Name            1590 non-null   object        
 1   City            1586 non-null   object        
 2   State           1586 non-null   object        
 3   Address         1590 non-null   object        
 4   isCOD           1590 non-null   bool          
 5   Date Placed     1590 non-null   datetime64[ns]
 6   Status          1590 non-null   object        
 7   IVR             1590 non-null   object        
 8   Remarks         919 non-null    object        
 9   Total           1590 non-null   int64         
 10  Date Delivered  1401 non-null   datetime64[ns]
 11  PID             1555 non-null   object        
 12  Category        1587 non-null   object        
 13  Quantity        1590 non-null   int64         
 14  Product Name    1590 non-null   object        
dtypes: b

In [32]:
# set index to date placed
df_datetime = df_changed.set_index("Date Placed")
df_datetime.head()

,Name,City,State,Address,isCOD,Status,IVR,Remarks,Total,Date Delivered,PID,Category,Quantity,Product Name
Date Placed,,,,,,,,,,,,,,
2021-01-31 17:45:59,Man,Nayagarh,Odisha,"Jagadish Prasad, Golapokhari-Bhapur Road, Fate...",True,Delivered,Confirmed,pickrr,999,2021-02-14 04:37:00,32,WL,2,One Week Weight-Loss (Peach)
2021-01-31 17:33:57,Dik,Thane,Maharashtra,"Shanti Garden Sector 4 , flat 401, building 2,...",True,Delivered,Confirmed,NaN,599,2021-02-02 23:19:18,23,D,2,One Week Detox Trial
2021-01-31 17:33:02,Shi,Bangalore,Karnataka,"#280 2nd main 2nd cross, vinobha nagar kg halli",True,Delivered,Confirmed,On call,599,2021-02-08 00:08:25,23,D,2,One Week Detox Trial
2021-01-31 17:31:57,Pre,Mumbai,Maharashtra,"1404/ Accord Nidhi, above Bikaji, Link Road, M...",True,Delivered,Confirmed,Script,3596,2021-02-09 03:17:08,34,WL,2,One Month Weight-Loss (Peach)
2021-01-31 17:06:55,Dr.,Pauri Garhwal,Uttarakhand,"10 am to 3 pm, Please call Before Coming, Room...",True,Delivered,Confirmed,Script,999,2021-02-04 01:59:40,31,WL,2,One Week Weight-Loss (Mint)
